In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline

import warnings 
warnings.filterwarnings(action='ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
pd.options.display.float_format = '{:.0f}'.format


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/company_nps_data.csv', encoding='cp949')

In [ ]:
train

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151
...,...,...,...,...,...,...
99796,171439,4630622,2017,2,2402740,11
99797,239388,1045021,2018,10,1428800,9
99798,272304,181898,2018,3,392560,4
99799,362194,2100218,2018,12,874780,3


In [ ]:
# 결측치 확인
print(train.info())
print(len(train.groupby('회사ID')))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99801 entries, 0 to 99800
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   회사ID      99801 non-null  int64
 1   연매출액      99801 non-null  int64
 2   년도        99801 non-null  int64
 3   월         99801 non-null  int64
 4   월별_연금보험료  99801 non-null  int64
 5   월별_직원수    99801 non-null  int64
dtypes: int64(6)
memory usage: 4.6 MB
None
5000


In [ ]:
train['myear'] = pd.to_datetime(train['년도'].astype(str) + train['월'].astype(str), format='%Y%m').dt.strftime('%Y%m')

In [ ]:
print(mpl.__version__)
print(mpl.__file__)
print(mpl.get_configdir())
print(mpl.get_cachedir())

3.2.2
/usr/local/lib/python3.7/dist-packages/matplotlib/__init__.py
/root/.config/matplotlib
/root/.cache/matplotlib


# 상관 관계가 있는 항목
- 연매출액과 월별_직원수 
- 연매출액과 월별_연금보험료
- 월별_직원수와 월별_연금보험료

외에 상관관계는 없는것으로 판단

In [ ]:
### 직원수 증감률 및 매출액 증감률 계산
def make_increase_ratio(df):
  df['직원수증감률'] = df.groupby('회사ID')['월별_직원수'].pct_change() *100
  df['매출액증감률'] = df.groupby('회사ID')['연매출액'].pct_change() *100
  df['보험료증감률'] = df.groupby('회사ID')['월별_연금보험료'].pct_change() *100
  return df

In [ ]:
### 매출액 대비 인원수 컬럼 추가
def make_sales_employee_ratio(df):
  df['1인당매출액'] = (df['연매출액']/df['월별_직원수'])
  df['매출액대비연금료'] = (df['월별_연금보험료']/df['연매출액']) * 100
  return df

In [ ]:
pdf = make_increase_ratio(train)
pdf = make_sales_employee_ratio(pdf)
pdf= pdf.sort_values(['회사ID','myear'])

In [ ]:
pdf

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,myear,직원수증감률,매출액증감률,보험료증감률,1인당매출액,매출액대비연금료
33039,63,11328779396,2018,1,403119840,1006,201801,0,0,1,11261212,4
74251,63,11328779396,2018,2,406770960,1018,201802,0,0,-3,11128467,4
80947,63,11328779396,2018,3,404826240,1013,201803,0,0,0,11183395,4
80770,63,11328779396,2018,4,404211800,1012,201804,1,0,-3,11194446,4
28110,63,11328779396,2018,5,400339640,1002,201805,1,0,-3,11306167,4
...,...,...,...,...,...,...,...,...,...,...,...,...
244,3312354,1451563,2019,9,1312200,3,201909,nan,nan,nan,483854,90
38757,3312354,1451563,2019,10,1312200,3,201910,0,0,0,483854,90
49019,3312354,1451563,2019,11,1312200,3,201911,0,0,0,483854,90
72163,3312354,1451563,2019,12,1312200,3,201912,0,0,0,483854,90


In [ ]:
# null값 확인
pdf.isnull().sum()

회사ID           0
연매출액           0
년도             0
월              0
월별_연금보험료       0
월별_직원수         0
myear          0
직원수증감률      5002
매출액증감률      5000
보험료증감률      5002
1인당매출액         0
매출액대비연금료       0
dtype: int64

In [ ]:
# 월별데이터 확인
pdf.groupby('회사ID')['년도'].value_counts()

회사ID     년도  
63       2018    12
95       2018    12
102      2018     8
295      2018     8
414      2018    12
                 ..
2400284  2018     7
2437574  2018     1
2693684  2019     6
3312354  2019     5
5159723  2019     1
Name: 년도, Length: 10111, dtype: int64

In [ ]:
pdf['연매출액'].describe()
pdf.sort_values(['회사ID', '연매출액'])

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,myear,직원수증감률,매출액증감률,보험료증감률,1인당매출액,매출액대비연금료
33039,63,11328779396,2018,1,403119840,1006,201801,0,0,1,11261212,4
74251,63,11328779396,2018,2,406770960,1018,201802,0,0,-3,11128467,4
80947,63,11328779396,2018,3,404826240,1013,201803,0,0,0,11183395,4
80770,63,11328779396,2018,4,404211800,1012,201804,1,0,-3,11194446,4
28110,63,11328779396,2018,5,400339640,1002,201805,1,0,-3,11306167,4
...,...,...,...,...,...,...,...,...,...,...,...,...
244,3312354,1451563,2019,9,1312200,3,201909,nan,nan,nan,483854,90
38757,3312354,1451563,2019,10,1312200,3,201910,0,0,0,483854,90
49019,3312354,1451563,2019,11,1312200,3,201911,0,0,0,483854,90
72163,3312354,1451563,2019,12,1312200,3,201912,0,0,0,483854,90


In [ ]:
### nan 값이 있는 곳 확인 => 직원수 증감률 & 매출액 증감률의 경우 첫달은 측정 불가능
print(np.where(pdf.isna()))
## 연매출액 마이너스 확인
print(pdf.loc[pdf['연매출액']< 0, '회사ID'].unique())
minus_ma = []
minus_ma = pdf.loc[pdf['연매출액']< 0, '회사ID'].unique()
pdf = pdf[~pdf['회사ID'].isin(minus_ma)]

# 직원수 0 제외
print(np.where(pdf['월별_직원수']==0))
no_emp = []
no_emp = np.where(pdf['월별_직원수']==0)[0]
pdf = pdf[~pdf['회사ID'].isin(no_emp)]

# 3년(36개월)치 자료가 없는 회사 조회
grouped = pdf.groupby('회사ID')
delist = []
delist = grouped.filter(lambda g : len(g) < 36)['회사ID'].unique()
pdf = pdf[~pdf['회사ID'].isin(delist)]

# 연매출액 249억 이상 기업만 
ma_list = []
ma_list = list(set(pdf[pdf['연매출액'] > 24900000]['회사ID'].unique()))
rdf = pdf[pdf['회사ID'].isin(ma_list)]
rdf = rdf.sort_values(['회사ID','myear'], axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=True, key=None)

(array([    4,     4,     4, ..., 42398, 42398, 42398]), array([7, 8, 9, ..., 7, 8, 9]))
[]
(array([], dtype=int64),)


In [ ]:
len(rdf['회사ID'].unique())

288

In [ ]:
rdf= make_increase_ratio(rdf)

In [ ]:
rdf = make_sales_employee_ratio(rdf)

In [ ]:
rdf

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,myear,직원수증감률,매출액증감률,보험료증감률,1인당매출액,매출액대비연금료
0,122968,37415377,2015,11,18477180,88,201511,nan,nan,nan,425175,49
1,122968,37415377,2015,12,18424620,88,201512,0,0,-0,425175,49
2,122968,26858719,2016,1,15053500,73,201601,-17,-28,-18,367928,56
3,122968,26858719,2016,2,14901760,72,201602,-1,0,-1,373038,55
4,122968,26858719,2016,3,15056200,73,201603,1,0,1,367928,56
...,...,...,...,...,...,...,...,...,...,...,...,...
10962,510449,28299495,2019,8,30448620,152,201908,-1,0,-1,186181,108
10963,510449,28299495,2019,9,30440700,154,201909,1,0,-0,183763,108
10964,510449,28299495,2019,10,26208980,127,201910,-18,0,-14,222831,93
10965,510449,28299495,2019,11,25320800,123,201911,-3,0,-3,230077,89


In [ ]:
print(len(rdf['회사ID'].unique()))
print(rdf['회사ID'].unique())

288
[122968 125878 126521 126538 126606 126664 126674 126772 126802 126814
 126831 126983 127060 127065 127090 127142 127180 127202 127366 127503
 127511 127669 127732 127839 128076 128149 128174 128264 128405 128415
 128440 128485 128488 128539 128638 128723 128896 128916 129184 129310
 129370 129377 129436 129486 129652 129718 129828 129861 130023 130084
 130128 130191 130203 130281 130322 130344 130386 130387 130577 130647
 130729 130899 130911 130927 130966 131049 131114 131228 131293 131345
 131397 131440 131485 131635 131723 131872 131916 132056 132156 132164
 132219 132335 132345 132359 132623 132760 132930 133019 133175 133298
 133431 133493 133681 133739 133826 133951 134484 134905 135061 135634
 135668 135798 136174 136566 137166 137377 137497 137916 137988 138077
 138269 138433 138591 138709 139080 139359 139393 139414 140574 141095
 141220 142535 142762 143050 143187 144894 145177 146654 148008 148537
 149366 150455 152219 152490 153669 154500 155754 157470 158094 227414
 2

In [ ]:
edf = pd.DataFrame(rdf.groupby(['회사ID','년도'])['월별_직원수'].sum()/rdf.groupby(['회사ID','년도'])['월별_직원수'].count())

In [ ]:
edf = edf.rename(columns={'월별_직원수':'연평균_직원수'})
edf

연평균_직원수
회사ID   년도           
122968 2015       88
       2016       76
       2017       71
       2018       66
125878 2016       27
...              ...
510329 2017      593
       2018      582
510449 2016       64
       2017       88
       2019      145

[1129 rows x 1 columns]

In [ ]:
mdf= pd.merge(rdf, edf, left_on = ['회사ID','년도'], right_on=['회사ID','년도'],how = 'left')
mdf

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,myear,직원수증감률,매출액증감률,보험료증감률,1인당매출액,매출액대비연금료,연평균_직원수
0,122968,37415377,2015,11,18477180,88,201511,nan,nan,nan,425175,49,88
1,122968,37415377,2015,12,18424620,88,201512,0,0,-0,425175,49,88
2,122968,26858719,2016,1,15053500,73,201601,-17,-28,-18,367928,56,76
3,122968,26858719,2016,2,14901760,72,201602,-1,0,-1,373038,55,76
4,122968,26858719,2016,3,15056200,73,201603,1,0,1,367928,56,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10962,510449,28299495,2019,8,30448620,152,201908,-1,0,-1,186181,108,145
10963,510449,28299495,2019,9,30440700,154,201909,1,0,-0,183763,108,145
10964,510449,28299495,2019,10,26208980,127,201910,-18,0,-14,222831,93,145
10965,510449,28299495,2019,11,25320800,123,201911,-3,0,-3,230077,89,145


In [ ]:
flist =rdf.drop_duplicates(['회사ID','년도'])['연매출액'].tolist()
# flist

In [ ]:
edf.index ## multi_index
edf= edf.reset_index(level = [0,1]) ## multi_index 풀기 위함 to single index
edf

,회사ID,년도,연평균_직원수
0,122968,2015,88
1,122968,2016,76
2,122968,2017,71
3,122968,2018,66
4,125878,2016,27
...,...,...,...
1124,510329,2017,593
1125,510329,2018,582
1126,510449,2016,64
1127,510449,2017,88


In [ ]:
edf['연매출액'] = flist

In [ ]:
edf['1인당매출액'] = edf['연매출액']/edf['연평균_직원수']
edf['1인당매출액_상승률'] = round(edf.groupby('회사ID')['1인당매출액'].pct_change(), 4) * 100
edf['연평균직원수_상승률'] = round(edf.groupby('회사ID')['연평균_직원수'].pct_change(), 4) * 100
edf

,회사ID,년도,연평균_직원수,연매출액,1인당매출액,1인당매출액_상승률,연평균직원수_상승률
0,122968,2015,88,37415377,425175,nan,nan
1,122968,2016,76,26858719,352631,-17,-13
2,122968,2017,71,32997924,463129,31,-6
3,122968,2018,66,31514468,473902,2,-7
4,125878,2016,27,23932243,891885,nan,nan
...,...,...,...,...,...,...,...
1124,510329,2017,593,134718002,227180,-0,-1
1125,510329,2018,582,143513683,246552,9,-2
1126,510449,2016,64,24450163,383031,nan,nan
1127,510449,2017,88,25154072,286113,-25,38


In [ ]:
elist = list(edf[edf['연평균직원수_상승률']<0]['회사ID'].unique())
len(elist)

198

In [ ]:
dlist = list(edf[edf['1인당매출액_상승률']<0]['회사ID'].unique())
len(dlist)

217

In [ ]:
ldf = edf[~edf['회사ID'].isin(dlist)]
ldf = ldf[~ldf['회사ID'].isin(elist)]

In [ ]:
ldf

,회사ID,년도,연평균_직원수,연매출액,1인당매출액,1인당매출액_상승률,연평균직원수_상승률
19,126664,2015,742,256505233,345694,nan,nan
20,126664,2016,782,304408150,389103,13,5
21,126664,2017,818,323278620,395126,2,5
22,126664,2018,845,353839107,418579,6,3
71,127366,2015,244,55796793,228208,nan,nan
...,...,...,...,...,...,...,...
1070,440094,2018,344,43950506,127825,0,22
1106,469677,2015,344,48624183,141349,nan,nan
1107,469677,2016,354,60309282,170205,20,3
1108,469677,2017,425,83821545,197111,16,20


In [ ]:
len(ldf.회사ID.unique())

23

In [ ]:
ldf['연매출액_상승률'] = round(ldf.groupby('회사ID')['연매출액'].pct_change(), 4)*100

In [ ]:
ldf

,회사ID,년도,연평균_직원수,연매출액,1인당매출액,1인당매출액_상승률,연평균직원수_상승률,연매출액_상승률
19,126664,2015,742,256505233,345694,nan,nan,nan
20,126664,2016,782,304408150,389103,13,5,19
21,126664,2017,818,323278620,395126,2,5,6
22,126664,2018,845,353839107,418579,6,3,9
71,127366,2015,244,55796793,228208,nan,nan,nan
...,...,...,...,...,...,...,...,...
1070,440094,2018,344,43950506,127825,0,22,22
1106,469677,2015,344,48624183,141349,nan,nan,nan
1107,469677,2016,354,60309282,170205,20,3,24
1108,469677,2017,425,83821545,197111,16,20,39


In [ ]:
percent = 100
print( '1인당 매출액 상승률 {}% 이상인 기업은 {} 곳'.format(percent,len(ldf[ldf['1인당매출액_상승률']>percent]['회사ID'].unique())))
print( '연매출액 상승률 {}% 이상인 기업은 {} 곳'.format(percent,len(ldf[ldf['연매출액_상승률']>percent]['회사ID'].unique())))
print( '연평균직원수 상승률 {}% 이상인 기업은 {} 곳'.format(percent,len(ldf[ldf['연평균직원수_상승률']>percent]['회사ID'].unique())))
print( '1인당 매출액 상승률{}% 이상인 기업은 {} 곳'.format(30,len(ldf[ldf['1인당매출액_상승률']>30]['회사ID'].unique())))
print( '연매출액 상승률 {}% 이상인 기업은 {} 곳'.format(30,len(ldf[ldf['연매출액_상승률']>30]['회사ID'].unique())))
print( '연평균직원수 상승률 {}% 이상인 기업은 {} 곳'.format(30,len(ldf[ldf['연평균직원수_상승률']>30]['회사ID'].unique())))
print( '연매출액{} 이상인 기업은 {} 곳'.format(100000000,len(ldf[ldf['연매출액']>100000000]['회사ID'].unique())))
print( '연매출액{} 이상인 기업은 {} 곳'.format(24900000,len(ldf[ldf['연매출액']>24900000]['회사ID'].unique())))

### 1인당 매출액 상승률이 100 % 가 넘었던 기업이 8곳, 
### 연매출액 상승률 100 % 가 넘었던 기업도 13곳.
### 이를 통해, 1인당 매출액 상승률 혹은 연매출액 상승률이 단기간에 급상승한 경우도 고려해볼 가능성이 있을 것으로 보임.

1인당 매출액 상승률 100% 이상인 기업은 2 곳
연매출액 상승률 100% 이상인 기업은 4 곳
연평균직원수 상승률 100% 이상인 기업은 1 곳
1인당 매출액 상승률30% 이상인 기업은 9 곳
연매출액 상승률 30% 이상인 기업은 12 곳
연평균직원수 상승률 30% 이상인 기업은 3 곳
연매출액100000000 이상인 기업은 9 곳
연매출액24900000 이상인 기업은 23 곳


In [ ]:
len(ldf['회사ID'].unique())

23

In [ ]:
fin_df = ldf

In [ ]:
fin_df = fin_df.sort_values('1인당매출액_상승률',  ascending=False)

In [ ]:
fin_df['회사ID'].unique()[:10]

array([127366, 133493, 404804, 420165, 132335, 128405, 306768, 318919,
       420304, 321054])

In [ ]:
 ## 연매출액 200억 및 연매출액 상승률 30 % 이상인 기업으로 필터링
24900000
ma_list = list(ldf[ldf['연매출액'] > 24900000]['회사ID'].unique())
# inc_list = list(ldf[ldf['연매출액_상승률']>30]['회사ID'].unique())
# inc_list
idf = ldf[ldf['회사ID'].isin(ma_list)]
# idf = idf[idf['회사ID'].isin(inc_list)]

In [ ]:
len(list(set(m_list)))

36

In [ ]:
idf

,회사ID,년도,연평균_직원수,연매출액,1인당매출액,1인당매출액_상승률,연평균직원수_상승률,연매출액_상승률
84,126664,2015,742,256505233,345694,nan,nan,nan
85,126664,2016,782,304408150,389103,13,5,19
86,126664,2017,818,323278620,395126,2,5,6
87,126664,2018,845,353839107,418579,6,3,9
136,127366,2015,244,55796793,228208,nan,nan,nan
...,...,...,...,...,...,...,...,...
3808,440094,2018,344,43950506,127825,0,22,22
4026,469677,2015,344,48624183,141349,nan,nan,nan
4027,469677,2016,354,60309282,170205,20,3,24
4028,469677,2017,425,83821545,197111,16,20,39


In [ ]:
# 당해 1인당 매출액 상승률이 그 전 해의 매출액 상승률보다 큰 경우
me_list =[]
for i in range(len(idf['회사ID'])):
# print(idf.iloc[i, 5]
  if i > 1:
    if idf.iloc[i, 5] < idf.iloc[i-1, 5]:
      me_list.append(idf.iloc[i, 0])

In [ ]:
udf = idf[~idf['회사ID'].isin(me_list)]
udf

,회사ID,년도,연평균_직원수,연매출액,1인당매출액,1인당매출액_상승률,연평균직원수_상승률,연매출액_상승률
303,130344,2015,92,45631762,493316,nan,nan,nan
304,130344,2016,100,52047832,523094,6,8,14
305,130344,2017,105,59854757,568241,9,6,15
306,130344,2018,118,74511742,632349,11,12,24
452,132760,2015,72,31090134,431807,nan,nan,nan
453,132760,2016,76,33268962,440649,2,5,7
454,132760,2017,76,36023185,470891,7,1,8
455,132760,2018,77,39290729,508070,8,1,9
495,133493,2015,45,9754970,216777,nan,nan,nan
496,133493,2016,50,13806380,273846,26,12,42


In [ ]:
print('연매출액 249억, 연매출액 상승률 30프로 이상 기업은 {}개'.format(len(idf['회사ID'].unique())))
print('연매출액 249억, 연매출액 상승률 30프로 이상 기업 중, 당해 년도 1인당 매출액 상승률이 전년대비 항상 큰 기업 {} 개'.format(len(udf['회사ID'].unique())))

연매출액 249억, 연매출액 상승률 30프로 이상 기업은 23개
연매출액 249억, 연매출액 상승률 30프로 이상 기업 중, 당해 년도 1인당 매출액 상승률이 전년대비 항상 큰 기업 7 개


In [ ]:
idf['회사ID'].unique()

array([126664, 127366, 128405, 130344, 131872, 132335, 132760, 132930,
       133493, 227414, 227415, 294337, 294530, 301743, 306768, 318919,
       321054, 404804, 420046, 420165, 420304, 440094, 469677])

In [ ]:
idf.sort_values('1인당매출액_상승률')['회사ID'].unique()[:10]

array([440094, 318919, 469677, 227415, 301743, 132930, 126664, 132760,
       227414, 128405])

In [ ]:
list1 = [133493, 127366, 420165, 404804, 306768, 128405, 132335, 420304, 294337, 318919]
list2 = [127366, 133493, 404804, 420165, 132335, 128405, 306768, 318919, 420304, 321054]

list1 == list2

False

In [236]:
count = 0
for i in list1:
  if i in list2:
    count += 1
print(count)

9
